Medium query should have from 2 to 3 tables joined and use built-in SQL functions and group by summarization. It should include combinations of subqueries or CTE or virtual tables.  
  
8 Medium Queries

**Medium Prompt 1:**

WideWorldImporters is interested to determine the top 3 suppliers for the last year based on the total quantity of products they supplied. Alongside this, find the average payment days associated with these suppliers. This will help us understand who our major suppliers are and the average credit terms they offer.

**Database :** WideWorldImportersDW 

**Tables Used :** Fact.Purchase, Dimension.Supplier,

**CTE :** TopSuppliers

In [22]:
-- Using the WideWorldImportersDW database
USE WideWorldImportersDW;

-- Define a CTE to retrieve the top 3 suppliers based on total supplied quantity for the last fiscal year
WITH TopSuppliers AS (
    -- Retrieve the top 3 suppliers
    SELECT TOP(3)
        p.[Supplier Key],
        -- Aggregate the total ordered quantity for each supplier
        SUM(p.[Ordered Quantity]) as TotalSuppliedQuantity
    FROM
        Fact.Purchase p
    -- Join with the Date dimension to filter results based on the fiscal year
    JOIN Dimension.Date d ON p.[Date Key] = d.Date
    WHERE
        -- Filtering for the previous fiscal year
        d.[Fiscal Year] = (SELECT MAX([Fiscal Year]) - 1 FROM Dimension.Date)
    GROUP BY
        p.[Supplier Key]
    -- Sorting in descending order to get the top suppliers
    ORDER BY
        TotalSuppliedQuantity DESC
)

-- Main Query to retrieve supplier details along with the aggregated values from the CTE
SELECT
    s.Supplier,
    s.[Supplier Reference],
    ts.TotalSuppliedQuantity,
    -- Calculate the average payment days for each of these top suppliers
    AVG(s.[Payment Days]) as AveragePaymentDays
FROM
    TopSuppliers ts
-- Join the CTE with the Supplier dimension to get detailed information about the supplier
JOIN Dimension.Supplier s ON ts.[Supplier Key] = s.[Supplier Key]
GROUP BY
    s.Supplier,
    s.[Supplier Reference],
    ts.TotalSuppliedQuantity
-- Sorting by the total supplied quantity to list the top supplier first
ORDER BY
    ts.TotalSuppliedQuantity DESC;


(3 rows affected)

Total execution time: 00:00:00.029

Supplier,Supplier Reference,TotalSuppliedQuantity,AveragePaymentDays
"Fabrikam, Inc.",293092,30009348,30
"Litware, Inc.",BC0280982,19250561,30
A Datum Corporation,AA20384,5076,14


**Medium Prompt 2:**

WideWorldImports aims to find out the top 3 cities that has the highest number of suppliers, and for each city, we want to know the average payment days for those suppliers.

**Database :** WideWorldImporters 

**Tables Used :** Application.Cities, Application.StateProvinces, Purchsing.Suppliers, CitySupplier  
**CTE :**  CitySupplier

In [30]:
USE WideWorldImporters;
WITH CitySupplier AS (
    SELECT 
        c.CityID,
        c.CityName,
        sp.StateProvinceName,
        s.SupplierId,
        s.PaymentDays
    FROM Application.Cities AS c
    JOIN Application.StateProvinces AS sp ON c.StateProvinceID = sp.StateProvinceID
    JOIN Purchasing.Suppliers AS s ON c.CityID = s.DeliveryCityID
)

SELECT TOP(3)
    CityName,
    StateProvinceName,
    COUNT(SupplierId) AS NumberOfSuppliers,
    AVG(PaymentDays) AS AvgPaymentDays
FROM CitySupplier
GROUP BY CityName, StateProvinceName
ORDER BY NumberOfSuppliers DESC

-- CitySupplier is a CTE that holds CITY ID, CITY NAME, StateProvinceName
-- SupplierID, PaymentDays outsourced from 3 different tables.

-- We then use it on our main 
-- We first group each City and State together as one entity
-- We then count the unique suppliers
-- We then avg all entries of Paymentdays 




(3 rows affected)

Total execution time: 00:00:00.015

CityName,StateProvinceName,NumberOfSuppliers,AvgPaymentDays
San Francisco,California,2,18
Lakeview Heights,Kentucky,1,30
Zionsville,Indiana,1,14


**Medium Prompt 3:**

WideWorldImporters is interested in the total sales transactions for each customer, and identify which customers have had more than five transactions in the database. For each of those customers, display their name, the city they are located in, and the total number of transactions they have made.

**Database :** WideWorldImporters 

**Tables Used :** Application.Cities, Application.StateProvinces, Purchsing.Suppliers, CitySupplier

**CTE :** CustomerTransactionCount

In [33]:
USE WideWorldImporters;
WITH CustomerTransactionCount AS (
    SELECT 
        c.CustomerID,
        c.CustomerName,
        city.CityName,
        COUNT(ct.CustomerTransactionID) AS TotalTransactions
    FROM Sales.Customers AS c
    JOIN Sales.CustomerTransactions AS ct ON c.CustomerID = ct.CustomerID
    JOIN Application.Cities AS city ON c.DeliveryCityID = city.CityID
    GROUP BY c.CustomerID, c.CustomerName, city.CityName
)

SELECT 
    CustomerName,
    CityName,
    TotalTransactions
FROM CustomerTransactionCount
WHERE TotalTransactions > 5
ORDER BY TotalTransactions DESC;

-- We first create CustomerTransactionCount CTE
-- We're taking columns from Sales.CustomerTransaction, Application.Cities, and Sales.Customers
-- We then filter using the where clause 
-- to return all customers who has had more than 5 transactions


(263 rows affected)

Total execution time: 00:00:00.056

CustomerName,CityName,TotalTransactions
Tailspin Toys (Head Office),Lisco,23384
Wingtip Toys (Head Office),Orchard Mesa,23233
Bhaavan Rai,Chalco,283
Aleksandrs Riekstins,La Bolt,270
Anca Gogean,Arrowbear Lake,265
Bishwa Chatterjee,Lake Clarke Shores,264
Om Yadav,Lunds,263
Ingrida Zeltina,Trilby,261
Elina Kaleja,Golden Meadow,257
Shantanu Huq,Cramerton,256


**Medium Prompt 4:**

Prestige Cars wants to determine the total sales by country for a 2018 and compare it against the sales budget for that year. This requires joining tables Data.Sales, Data.Customer, and Reference.SalesBudgets.

**Database :** PrestigeCars

**Tables Used :** Data.Customer, Data.Sales

**CTE :**  SalesSummary

In [43]:
USE PrestigeCars;
WITH SalesSummary AS (
    SELECT 
        c.Country AS SalesCountry,
        SUM(s.TotalSalePrice) AS TotalSales,
        AVG(s.TotalSalePrice) AS AverageSale
    FROM 
        Data.Sales s
    JOIN 
        Data.Customer c ON s.CustomerID = c.CustomerID
    WHERE 
        YEAR(s.SaleDate) = 2018
    GROUP BY 
        c.Country
)

SELECT 
    ss.SalesCountry,
    ss.TotalSales,
    ss.AverageSale
FROM 
    SalesSummary ss
ORDER BY 
    ss.TotalSales DESC;

-- SalesSummary is a CTE composed of columns from Data.Sales and Data.Customer
-- The entries are grouped by so we only account for each country once
-- We then take the sum of the sales and the average of their transactions
-- We're specifically concerned for the year 2018 

-- We then select these specific items for our actual query 


(8 rows affected)

Total execution time: 00:00:00.015

SalesCountry,TotalSales,AverageSale
GB,2781360.00,51506.666666
FR,2074790.00,98799.523809
US,1163880.00,68463.529411
ES,788530.00,78853.000000
IT,549840.00,91640.000000
CH,296265.00,42323.571428
DE,111050.00,55525.000000
BE,7900.00,3950.000000


**Medium Prompt 5:**

Which 5 products have the highest total sales, grouped by product subcategory, and only consider sales that had a promotion applied. Display the top 5 subcategories.

**Database :** AdventureWorks2017DW

**Tables Used :** dbo.FactInternetSales, dbo.DimProduct, dbo.DimProductSubcategory

**CTE :** SalesWithPromotion

In [44]:
USE AdventureWorksDW2017;
WITH SalesWithPromotion AS (
    SELECT
        p.ProductSubcategoryKey AS SubcategoryKey,
        SUM(f.SalesAmount) AS TotalSales
    FROM
        dbo.FactInternetSales f
    JOIN
        dbo.DimProduct p ON f.ProductKey = p.ProductKey
    WHERE
        f.PromotionKey IS NOT NULL
    GROUP BY
        p.ProductSubcategoryKey
)

SELECT TOP (5)
    s.SubcategoryKey,
    sc.ProductSubcategoryAlternateKey,
    sc.ProductCategoryKey,
    s.TotalSales
FROM
    SalesWithPromotion s
JOIN
    dbo.DimProductSubcategory sc ON s.SubcategoryKey = sc.ProductSubcategoryKey
ORDER BY
    s.TotalSales DESC

-- We first identify all the products with a promotion by declaring a CTE
-- We're joining dbo.FactInternetSales, dbo.DimProduct and we're
-- We then group all the entries by ProductSubcategoryKey
-- filtering using PromotionKey IS NOT NULL, meaning a promotion is applied

-- In our main query we only want the top 5 
-- Now we want to find out which subcategory it is that's why 
-- We're using DimProductSubcategory with our CTE


(5 rows affected)

Total execution time: 00:00:00.119

SubcategoryKey,ProductSubcategoryAlternateKey,ProductCategoryKey,TotalSales
2,2,1,14520584.0363
1,1,1,9952759.5644
3,3,1,3844801.05
37,37,4,245529.32
31,31,4,225335.60


**Medium Prompt 6:**

Coroporate wants to identify the departments that hired during recruitment season. Retrieve departments, the number of employees in each department, and the latest hire date for those departments with more than 5 employees.

**Database :** AdventureWorks2017

**Tables Used :** HumanReasources.Department, HumanResources.EmployeeDepartmentHistory,  
HumanResources.Employee, DepartmentEmployee Count 

**CTE :** DepartmentEmployeeCount

In [45]:
USE AdventureWorks2017;
WITH DepartmentEmployeeCount AS (
    SELECT 
        d.DepartmentID,
        d.Name AS DepartmentName,
        COUNT(DISTINCT e.BusinessEntityID) AS EmployeeCount
    FROM HumanResources.Department d
    JOIN HumanResources.EmployeeDepartmentHistory edh ON d.DepartmentID = edh.DepartmentID
    JOIN HumanResources.Employee e ON edh.BusinessEntityID = e.BusinessEntityID
    WHERE edh.EndDate IS NULL
    GROUP BY d.DepartmentID, d.Name
    HAVING COUNT(DISTINCT e.BusinessEntityID) > 5
)

SELECT 
    dec.DepartmentName,
    dec.EmployeeCount,

    (SELECT MAX(e.HireDate) 
     FROM HumanResources.Employee e 
     WHERE e.BusinessEntityID IN (

        SELECT edh2.BusinessEntityID
        FROM HumanResources.EmployeeDepartmentHistory edh2 
        WHERE edh2.DepartmentID = dec.DepartmentID AND edh2.EndDate IS NULL
    )) AS LatestHireDate

FROM DepartmentEmployeeCount dec
ORDER BY dec.EmployeeCount DESC, LatestHireDate DESC;

-- We first want to know the DepartMentEmployeecount. We make a CTE
-- It contains columns from Department, EmployeeDepartmentHistory, Employee
-- The columns for this CTE are the individual Departments
-- and the number of appearance of that department as count of emloyees
-- We then use a Having clause since we're trying to filter an aggregate 
-- function

-- In our actual query we also perform a nested subquery.
-- the nested subquery returns the latest hiredate by using the IN operator
-- to check for membership in the LatestHireDate subquery.


(12 rows affected)

Total execution time: 00:00:00.076

DepartmentName,EmployeeCount,LatestHireDate
Production,179,2010-03-10
Sales,18,2013-05-30
Purchasing,12,2011-02-25
Finance,10,2009-03-08
Information Services,10,2009-02-23
Marketing,9,2011-02-14
Facilities and Maintenance,7,2010-03-07
Engineering,6,2011-01-18
Quality Assurance,6,2010-02-23
Production Control,6,2009-03-03


**Medium Prompt 7:**

Management needs query fetches total sales for each employee and the product they have sold the most (by quantity). The subquery within the WHERE clause is used to filter only the products with the maximum quantity for each order, effectively giving us the top product for each employee.

**Database :** Northwinds2022TSQLV7

**Tables Used :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Sales.[Order] , </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Sales.OrderDetail, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">HumanResources.Employee, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">EmployeeSales</span>

**CTE :** EmployeeSales

In [47]:
USE Northwinds2022TSQLV7;
WITH EmployeeSales AS (
    -- Calculate the total sales amount and most sold product for each employee
    SELECT
        e.EmployeeID,
        e.EmployeeLastName,
        e.EmployeeFirstName,
        SUM(od.UnitPrice * od.Quantity * (1 - od.DiscountPercentage/100)) AS TotalSales
    FROM 
        HumanResources.Employee e
        JOIN Sales.[Order] o ON e.EmployeeID = o.EmployeeID
        JOIN Sales.OrderDetail od ON o.OrderID = od.OrderID
    GROUP BY
        e.EmployeeID, e.EmployeeLastName, e.EmployeeFirstName
)

SELECT
    es.EmployeeLastName,
    es.EmployeeFirstName,
    es.TotalSales,
    p.ProductName AS MostSoldProduct,
    MAX(od.Quantity) AS QuantityOfMostSoldProduct
FROM
    EmployeeSales es
    JOIN Sales.[Order] o ON es.EmployeeID = o.EmployeeID
    JOIN Sales.OrderDetail od ON o.OrderID = od.OrderID
    JOIN Production.Product p ON od.ProductID = p.ProductID
WHERE 
    od.Quantity = (
        SELECT MAX(Quantity)
        FROM Sales.OrderDetail
        WHERE OrderID = o.OrderID
    )
GROUP BY
    es.EmployeeLastName,
    es.EmployeeFirstName,
    es.TotalSales,
    p.ProductName
ORDER BY
    es.TotalSales DESC;

-- EmployeeSales is a CTE that composes of columns from
-- HR.Employee, Sales.Order, Sales.OrderDetail
-- This CTE holds all the employee sales as we just group it by 
-- employee id, and full name

-- In our main query we then use this as our main table
-- it is we then join sales.order, sales.orderdetail, and product.product
-- we group the items by product, total sales, and employee full name
-- to extract productname, and quantity
-- Our where filter returns to us the maximum  value in quantity
-- returning the most sold item 



(449 rows affected)

Total execution time: 00:00:00.141

EmployeeLastName,EmployeeFirstName,TotalSales,MostSoldProduct,QuantityOfMostSoldProduct
Peled,Yael,250014.48396000000,Product ACRVI,80
Peled,Yael,250014.48396000000,Product APITJ,35
Peled,Yael,250014.48396000000,Product AQOKR,40
Peled,Yael,250014.48396000000,Product ASTMN,60
Peled,Yael,250014.48396000000,Product BIUDV,24
Peled,Yael,250014.48396000000,Product BLCAX,50
Peled,Yael,250014.48396000000,Product BWRLG,50
Peled,Yael,250014.48396000000,Product CBRRL,60
Peled,Yael,250014.48396000000,Product CKEDC,50
Peled,Yael,250014.48396000000,Product COAXA,20


**Medium Prompt 8:**

PrestigeCars wants us to calculate the total sales (considering discounts) by country for each car make, based on the sales from the Data.Sales, Data.Customer, and Data.Stock (which has the ModelID we can link to Data.Make).

**Database :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">PrestigeCars</span>

**Tables Used :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Data.Sales, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Data.Customer, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">SalesWithDiscount, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Data.Stock, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">Data.Make</span>  

**CTE :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">SalesWithDiscount</span>

In [48]:
USE PrestigeCars;
WITH SalesWithDiscount AS (
    -- Calculate the discounted sale price for each sale detail
    SELECT 
        SD.SalesID,
        SD.StockID,
        SD.SalePrice - SD.LIneItemDiscount AS DiscountedSalePrice
    FROM 
        Data.SalesDetails SD
)

-- Calculate total sales by country for each car Make
SELECT 
    C.Country,
    MK.MakeName,
    SUM(SWD.DiscountedSalePrice) AS TotalSalesByMake
FROM 
    Data.Sales S
JOIN Data.Customer C ON S.CustomerID = C.CustomerID
JOIN SalesWithDiscount SWD ON S.SalesID = SWD.SalesID
JOIN Data.Stock STK ON SWD.StockID = STK.StockCode
JOIN Data.Make MK ON STK.ModelID = MK.MakeID
GROUP BY 
    C.Country,
    MK.MakeName
ORDER BY 
    TotalSalesByMake DESC;

-- First we make a CTE to identify all identify all our sales that had discounts
-- we do this by selecting columns from data.SalesDetails
-- we then subtract SalePrice - LineItemDiscount 
-- The main query then:
--  1. Joins the Sales table to the Customer table to determine the country of each sale.
--  2. Utilizes the previously defined CTE to get the discounted sale price.
--  3. Joins to the Stock table to get the ModelID for each sale.
--  4. Joins to the Make table to determine the car make for each sale.

Warning: Null value is eliminated by an aggregate or other SET operation.

(72 rows affected)

Total execution time: 00:00:00.048

Country,MakeName,TotalSalesByMake
GB,Lamborghini,273450.00
FR,Mercedes,255450.00
US,Maybach,247550.00
FR,McLaren,234250.00
GB,Porsche,222300.00
GB,Cadillac,201400.00
US,Riley,174850.00
US,Cadillac,164890.00
FR,Lamborghini,154500.00
GB,Ferrari,140000.00
